<a href="https://colab.research.google.com/github/brendanpshea/database_sql/blob/main/Database_06_WritingData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction
When working with databases, one of the most fundamental tasks is **writing data to tables**. In SQL, this is accomplished using the `INSERT INTO` statement. This statement allows you to add new rows of data to a table, specifying the values for each column. Additionally, you can use **constraints** to enforce rules and maintain data integrity within your tables.

In this chapter, we will explore the process of writing data to SQL tables using the `INSERT INTO` statement. We will cover both single-row and multiple-row insertions, and discuss what happens when constraints are violated. We will also touch on related topics such as `DELETE FROM`, "soft delete," `UPDATE`, and **triggers**.

To make our examples more concrete, we will be working with a sample database based on Rolling Stone's Greatest Albums of All Time. We will create tables for artists and albums, populate them with data, and demonstrate various scenarios related to writing data to SQL tables.

By the end of this chapter, you will have a solid understanding of how to insert data into SQL tables, handle constraints, and perform common data manipulation operations. Let's dive in and start exploring the world of writing data to SQL tables!

## Creating Tables for Rolling Stone's Greatest Albums of All Time
For this chapter, we'll be creating and populating a database based on the 2023 version of Rollingstone Magazine's "500 greatest albums of all time".

Before we can start inserting data into our database, we need to create the necessary tables. In this case, we'll create two tables: Artist and Album. Let's define the structure of these tables and include some constraints to ensure data integrity.

In [60]:
%load_ext sql
%sql sqlite:///greatest.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [61]:
%%sql
DROP TABLE IF EXISTS Artist;
DROP TABLE IF EXISTS Album;
CREATE TABLE Artist (
    ArtistID INTEGER PRIMARY KEY AUTOINCREMENT,
    Name VARCHAR(100) NOT NULL,
    Country VARCHAR(50),
    Founded INT
    CHECK (Founded >= 1900 AND Founded <= 2023)
);

CREATE TABLE Album (
    AlbumID INTEGER PRIMARY KEY AUTOINCREMENT,
    Title VARCHAR(100) NOT NULL,
    ArtistID INT,
    ReleaseYear INT,
    Genre VARCHAR(50),
    Ranking,
    FOREIGN KEY (ArtistID) REFERENCES Artist(ArtistID),
    CHECK (ReleaseYear >= 1900 AND ReleaseYear <= 2023),
    CHECK (Ranking >= 1 AND Ranking <= 500)
);

--enable foreign key contraints
PRAGMA foreign_keys = ON;

 * sqlite:///greatest.db
Done.
Done.
Done.
Done.
Done.


[]

Let's break down the key aspects of these table definitions:

-   The `Artist` table has an `ArtistID` column as the **primary key**, which uniquely identifies each artist. The `Name` column is marked as `NOT NULL`, ensuring that every artist has a name. We also have `Country` and `Founded` columns to store additional information about the artists.
-   The `Album` table has an `AlbumID` column as the primary key. The `Title` column is marked as `NOT NULL` to ensure that every album has a title. We have a foreign key `ArtistID` that references the `ArtistID` column in the `Artist` table, establishing a relationship between albums and artists. The `ReleaseYear` and `Genre` columns provide additional details about each album.
-  We have specified that the primary keys (`ArtistID` and `AlbumID`) are both `NOT NULL` and `AUTOINCREMENT`. This means they cannot be left blank and that, if they are left blank, SQL will assign the next available integer to them.
-   We include **check constraints** in both tables to enforce certain conditions. In the `Artist` table, we ensure that the `Founded` year is between 1900 and 2023. Similarly, in the `Album` table, we check that the `ReleaseYear` is within the same range.
- Finally, we enable **foreign key constraints**, which means that SQLite will ensure that attempts to update, delete, or insert data that causes problems for this will fail.

By creating these tables with appropriate constraints, we set up a solid foundation for our database. The constraints help maintain data integrity by preventing invalid or inconsistent data from being inserted into the tables.

Now that we have our tables ready, we can start inserting data into them using the `INSERT INTO` statement, which we'll explore in the next section.

## Using INSERT INTO (Single Row)


Now that we have our `Artist` and `Album` tables created, let's explore how to insert a single row of data into each table using the `INSERT INTO` statement.

The basic syntax for inserting a single row is as follows:

```sql
INSERT INTO table_name (column1, column2, ...)
VALUES (value1, value2, ...);
```

Let's insert a single artist into the `Artist` table:

In [62]:
%%sql
--Insert Marvin Gaye
INSERT INTO Artist (ArtistID, Name, Country, Founded)
VALUES (1, 'Marvin Gaye', 'United States', 1939);

 * sqlite:///greatest.db
1 rows affected.


[]

In this example, we specify the table name `Artist` and list the columns we want to insert data into (`ArtistID`, `Name`, `Country`, `Founded`). We then provide the corresponding values for each column using the `VALUES` clause. The values are listed in the same order as the columns specified.

Now, let's insert a single album into the `Album` table:

In [63]:
%%sql
-- Insert What's Going On
INSERT INTO Album (AlbumID, Title, ArtistID, ReleaseYear,Genre, Ranking)
VALUES (1, 'What''s Going On', 1, 1971, 'Soul', 1);

 * sqlite:///greatest.db
1 rows affected.


[]

In [64]:
%sql SELECT * FROM Album;

 * sqlite:///greatest.db
Done.


AlbumID,Title,ArtistID,ReleaseYear,Genre,Ranking
1,What's Going On,1,1971,Soul,1


Similarly, we specify the `Album` table and list the columns we want to insert data into. We provide the corresponding values in the `VALUES` clause, ensuring that the `ArtistID` matches the `ArtistID` of the artist we inserted earlier.

It's important to note that when inserting data, we need to provide values for all columns that are marked as `NOT NULL` or do not have a default value defined. If we omit a column that allows `NULL` values or has a default value, the database will automatically assign the appropriate value.

By using the `INSERT INTO` statement, we can easily add single rows of data to our tables. This is particularly useful when we have specific values for each column and want to insert them one at a time.

In the next section, we'll explore how to insert multiple rows of data in a single statement, which is more efficient when dealing with larger datasets.

## Using INSERT INTO (Multiple Rows)

Inserting data one row at a time can be inefficient when you have a large number of records to insert. Fortunately, SQL allows you to insert multiple rows of data in a single `INSERT INTO` statement. This is achieved by specifying multiple sets of values in the `VALUES` clause.

Let's insert the top 10 albums from Rolling Stone's Greatest Albums of All Time list into our database. First, we'll insert the artists:

In [65]:
%%sql
INSERT INTO Artist (ArtistID, Name, Country, Founded)
VALUES
(2, 'The Beach Boys', 'United States', 1961),
(3, 'Joni Mitchell', 'Canada', 1964),
(4, 'Stevie Wonder', 'United States', 1961),
(5, 'Nirvana', 'United States', 1987),
(6, 'Fleetwood Mac', 'United Kingdom', 1967),
(7, 'Prince', 'United States', 1975),
(8, 'Bob Dylan', 'United States', 1961),
(9, 'Lauryn Hill', 'United States', 1988),
(10, 'The Beatles', 'United Kingdom', 1960),
(11, 'Radiohead', 'United Kingdom', 1985),
(12, 'Kendrick Lamar', 'United States', 2003),
(13, 'Public Enemy', 'United States', 1985),
(14, 'The Rolling Stones', 'United Kingdom', 1962),
(15, 'Aretha Franklin', 'United States', 1956),
(16, 'Michael Jackson', 'United States', 1964),
(17, 'Kanye West', 'United States', 1996),
(19, 'The Clash', 'United Kingdom', 1976);

 * sqlite:///greatest.db
17 rows affected.


[]

Now, let's insert the corresponding albums:

In [66]:
%%sql
INSERT INTO Album (AlbumID, Title, ArtistID, ReleaseYear, Genre, Ranking)
VALUES
(2, 'Pet Sounds', 2, 1966, 'Rock',2),
(3, 'Blue', 3, 1971, 'Folk',3),
(4, 'Songs in the Key of Life', 4, 1976, 'Soul',4),
(5, 'Nevermind', 5, 1991, 'Grunge',5),
(6, 'Rumours', 6, 1977, 'Soft Rock',6),
(7, 'Purple Rain', 7, 1984, 'Pop',7),
(8, 'Blood on the Tracks', 8, 1975, 'Folk Rock',8),
(9, 'The Miseducation of Lauryn Hill', 9, 1998, 'Hip Hop',9),
(10, 'Abbey Road', 10, 1969, 'Rock',10),
(11, 'Revolver', 10, 1966, 'Rock',11),
(12, 'Thriller', 16, 1982, 'Pop',12),
(13, 'I Never Loved a Man the Way I Love You', 15, 1967, 'Soul',13),
(14, 'Exile on Main Street', 14, 1972, 'Rock',14),
(15, 'It Takes a Nation of Millions to Hold Us Back', 13, 1988, 'Hip Hop',15),
(16, 'London Calling', 19, 1979, 'Punk',16),
(17, 'My Beautiful Dark Twisted Fantasy', 17, 2010, 'Hip Hop',17),
(18, 'Highway 61 Revisited', 8, 1965, 'Folk Rock',18),
(19, 'To Pimp a Butterfly', 12, 2015, 'Hip Hop',19),
(20, 'Kid A', 11, 2000, 'Electronic',20);

 * sqlite:///greatest.db
19 rows affected.


[]

In these examples, we use a single `INSERT INTO` statement for each table, but we provide multiple sets of values separated by commas. Each set of values represents a new row to be inserted into the table.

By inserting multiple rows at once, we can significantly reduce the number of statements required and improve the efficiency of our data insertion process.

It's important to ensure that the number of values matches the number of columns specified and that the values adhere to any constraints defined on the table. If there are any violations, such as trying to insert a duplicate primary key value or violating a check constraint, the entire `INSERT INTO` statement will fail, and no rows will be inserted.

In the next section, we'll explore what happens when constraints are violated and how to handle such situations.

## What Happens When Constraints Are Violated?

Constraints are essential for maintaining data integrity in a database. They enforce rules and restrictions on the data that can be inserted into tables. When attempting to insert data that violates these constraints, SQL will raise an error and prevent the insertion from occurring. Let's explore some common constraint violations and their consequences.

### Problems with Primary Keys

A primary key uniquely identifies each row in a table. It ensures that no two rows have the same primary key value. If we attempt to insert a row with a primary key value that already exists in the table, SQL will raise a primary key constraint violation error.

For example, let's try to insert an artist with an existing `ArtistID`:

In [67]:
%%sql
INSERT INTO Artist (ArtistID, Name, Country, Founded)
VALUES (1, 'Pink Floyd', 'United Kingdom', 1962);

 * sqlite:///greatest.db
(sqlite3.IntegrityError) UNIQUE constraint failed: Artist.ArtistID
[SQL: INSERT INTO Artist (ArtistID, Name, Country, Founded)
VALUES (1, 'Pink Floyd', 'United Kingdom', 1962);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


Since we already have an artist with an `artist_id` of 1 this fails with an error message. This is because primary keys must be UNIQUE.

### Violating CHECK Constraints

CHECK constraints allow us to specify conditions that the data must satisfy before it can be inserted into a table. If we attempt to insert data that violates a CHECK constraint, SQL will raise a constraint violation error.

For example, let's try to insert an album with an invalid `ReleaseYear`:

In [68]:
%%sql
INSERT INTO Album (AlbumID, Title, ArtistID, ReleaseYear, Genre)
VALUES (11, 'Future Album', 1, 2028, 'Rock');

 * sqlite:///greatest.db
(sqlite3.IntegrityError) CHECK constraint failed: ReleaseYear >= 1900 AND ReleaseYear <= 2023
[SQL: INSERT INTO Album (AlbumID, Title, ArtistID, ReleaseYear, Genre)
VALUES (11, 'Future Album', 1, 2028, 'Rock');]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In our Album table, we have a CHECK constraint that ensures the ReleaseYear is between 1900 and 2023. Since 2028 violates this constraint, the insertion will fail with an error message indicating a CHECK constraint violation.

### Violating NOT NULL Constraints

NOT NULL constraints ensure that a column cannot contain a NULL value. If we attempt to insert a row with a NULL value for a column that has a NOT NULL constraint, SQL will raise a constraint violation error.

For example, let's try to insert an artist without specifying a `Name`:

In [69]:
%%sql
INSERT INTO Artist (ArtistID, Country, Founded)
VALUES (21, 'United States', 1980);

 * sqlite:///greatest.db
(sqlite3.IntegrityError) NOT NULL constraint failed: Artist.Name
[SQL: INSERT INTO Artist (ArtistID, Country, Founded)
VALUES (21, 'United States', 1980);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


Since the `Name` column in the `Artist` table has a NOT NULL constraint, this insertion will fail with an error message indicating a NOT NULL constraint violation.

When constraint violations occur, the entire `INSERT INTO` statement is rolled back, and no data is inserted into the table. This ensures that the database remains in a consistent state and maintains data integrity.

To handle constraint violations, you can:

-   Modify the data being inserted to satisfy the constraints.
-   Update the table structure or constraints to accommodate the data.
-   Catch and handle the specific error messages in your application code.

By understanding and properly handling constraint violations, you can ensure that only valid and consistent data is inserted into your database tables.

### Auto-incrementing Primary Keys
As we just saw, SQL will generally produce an error if you fail to provide values for a `NOT NULL` column. One exception to this is for integer primary keys with an `AUTOINCREMENT` option (such as `ArtistId` and `AlbumId`). If you leave this out of an insert statement, SQL will assign the *next available integer as a primary key.

Here's an example:


In [70]:
%%sql
--Inserting without specifying primary key
INSERT INTO Artist (Name, Country, Founded)
VALUES ('Pink Floyd', 'United Kingdom', 1962);

 * sqlite:///greatest.db
1 rows affected.


[]

In [71]:
%%sql
SELECT *
FROM Artist
WHERE Name = "Pink Floyd";

 * sqlite:///greatest.db
Done.


ArtistID,Name,Country,Founded
20,Pink Floyd,United Kingdom,1962


## Deleting Data

When working with databases, there may be situations where you need to remove data from tables. In SQL, the `DELETE` statement is used to delete rows from a table based on specified conditions. It's important to understand how to use the `DELETE` statement effectively and handle the deletion of related data to maintain data integrity.

Let's start by inserting some sample data into our `Artist` and `Album` tables. We'll insert a fake artist named "The Terrible Trio" and a few terrible albums associated with this artist.

First, let's insert the fake artist into the `Artist` table:

In [72]:
%%sql
INSERT INTO Artist (Name, Country, Founded)
VALUES ('The Terrible Trio', 'Nowhere', 2020);

 * sqlite:///greatest.db
1 rows affected.


[]

Now, let's insert some terrible albums. We'll use a subquery to find the ArtistID for the "Terrible Trio."

In [73]:
%%sql
-- Inserting albums
INSERT INTO Album (Title, ArtistID, ReleaseYear, Genre)
VALUES
    ('Awful Anthems', (SELECT ArtistID FROM Artist WHERE Name = 'The Terrible Trio'), 2021, 'Noise'),
    ('Cringeworthy Chronicles', (SELECT ArtistID FROM Artist WHERE Name = 'The Terrible Trio'), 2022, 'Cacophony'),
    ('Disastrous Ditties', (SELECT ArtistID FROM Artist WHERE Name = 'The Terrible Trio'), 2023, 'Racket');

 * sqlite:///greatest.db
3 rows affected.


[]

Let's assure ourselves that these albums have been added.

In [74]:
%%sql
SELECT
  al.Title,
  al.AlbumID,
  ar.Name AS Artist,
  ar.ArtistID
FROM
  Album al
  JOIN Artist ar ON al.ArtistID = ar.ArtistID
WHERE ar.Name = 'The Terrible Trio';

 * sqlite:///greatest.db
Done.


Title,AlbumID,Artist,ArtistID
Awful Anthems,21,The Terrible Trio,21
Cringeworthy Chronicles,22,The Terrible Trio,21
Disastrous Ditties,23,The Terrible Trio,21


### Basic DELETE Statement

The basic syntax of the `DELETE` statement is as follows:

```sql
DELETE FROM table_name
WHERE condition;
```

-   `table_name`: The name of the table from which you want to delete rows.
-   `condition`: Specifies the condition that determines which rows will be deleted. If omitted, all rows in the table will be deleted.

For example, to delete the album "Awful Anthems" from the `Album` table, you can use the following statement:

In [75]:
%%sql
DELETE FROM Album
WHERE Title = 'Awful Anthems';

 * sqlite:///greatest.db
1 rows affected.


[]

In [76]:
%%sql
--Check that album is deleted
SELECT * FROM Album WHERE Title = 'Awful Anthems';

 * sqlite:///greatest.db
Done.


AlbumID,Title,ArtistID,ReleaseYear,Genre,Ranking


### Deleting Related Data

When deleting data from a table that has related data in other tables, you need to consider the foreign key constraints and how to handle the deletion of related records.

In our example, the `Album` table has a foreign key constraint on the `ArtistID` column that references the `ArtistID` column in the `Artist` table. This means that each album is associated with an artist.

In [77]:
%%sql
-- This will fail
DELETE FROM Artist
WHERE Name = "The Terrible Trio";

 * sqlite:///greatest.db
(sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: -- This will fail
DELETE FROM Artist
WHERE Name = "The Terrible Trio";]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


In order to deal with with this problem, you have a few different options.



### Manual Deletion
The most straightforward to way to do this deletion is to:
1. Delete the related albums from the Album table first.
2. Second, delete the artist from the Artist table.

```sql
--Delete albums
DELETE FROM Album WHERE ArtistID = (SELECT ArtistID FROM Artist WHERE Name = 'The Terrible Trio');

--Delete artist
DELETE FROM Artist WHERE Name = 'The Terrible Trio';
```

### ON DELETE CASCADE

The `ON DELETE CASCADE` option is used to automatically delete related records from a child table when a record in the parent table is deleted. It ensures data consistency and maintains referential integrity by cascading the delete operation to the associated records.

When you define a foreign key constraint with `ON DELETE CASCADE`, deleting a record from the parent table will automatically delete all the related records in the child table that reference the deleted parent record.

To set up `ON DELETE CASCADE`, you include it in the foreign key constraint definition when creating the child table:

```sql
CREATE TABLE Artist(
  -- define all your columms
  ...
  FOREIGN KEY (ArtistID) REFERENCES Artist(ArtistID) ON DELETE CASCADE
)
```

By adding this line to the foreign key constraint, you instruct the database to cascade the delete operation from the parent table (`Artist`) to the child table (`Album`) when an artist is deleted.

With this option set, you can delete an artist and its related albums using a single DELETE statement:

```sql
DELETE FROM Artist WHERE Name = 'The Terrible Trio';
```

### ON DELETE SET NULL

The `ON DELETE SET NULL` option is used to automatically set the foreign key values in the child table to `NULL` when a record in the parent table is deleted. It allows you to maintain the child records even if the associated parent record is removed.

When you define a foreign key constraint with `ON DELETE SET NULL`, deleting a record from the parent table will set the foreign key values in the child table to `NULL` for the related records.

To set up `ON DELETE SET NULL`, you include it in the foreign key constraint definition when creating the child table:

```sql
CREATE TABLE Artist(
  -- define all your columms
  ...
  FOREIGN KEY (ArtistID) REFERENCES Artist(ArtistID) ON DELETE SET NULL
)

```
By adding this line to the foreign key constraint, you instruct the database to set the `ArtistID` values in the `Album` table to `NULL` when an artist is deleted from the `Artist` table.

### Choosing Between ON DELETE CASCADE, ON DELETE SET NULL, and no action

The choice between `ON DELETE CASCADE` and `ON DELETE SET NULL` depends on your application's requirements and data integrity rules.

-   Use `ON DELETE CASCADE` when you want to ensure that related records in the child table are automatically deleted when a parent record is deleted. This option maintains strict referential integrity and removes all associated data.
-   Use `ON DELETE SET NULL` when you want to keep the related records in the child table even if the parent record is deleted. This option allows you to maintain the child records but sets the foreign key values to `NULL`, indicating that they are no longer associated with a valid parent record.

It's important to consider the implications of each option. `ON DELETE CASCADE` permanently deletes related data, which may not be desirable in all scenarios. `ON DELETE SET NULL` keeps the related records but with a `NULL` foreign key value, which may require additional handling in your application.

If you don't specify either `ON DELETE CASCADE` or `ON DELETE SET NULL`, the default behavior is to restrict the deletion of a parent record if there are related records in the child table. In this case, you would need to manually delete the related records from the child table before deleting the parent record.

### Deleting All Data versus Dropping a Table

As we've seen, the **`DELETE FROM`** command is used to remove rows from a table. This operation can be selective or comprehensive. For example, if you only want to delete rows that meet certain criteria, you use a **`WHERE`** clause with your **`DELETE`** statement. Without a **`WHERE`** clause, **`DELETE`** will remove all rows in the table, but importantly, the table's structure remains untouched.  So, If you decide to remove all albums from the **`albums`** table, you simply omit the **`WHERE`** clause:

```sql
DELETE FROM Albums;
```

This action clears all data from the **`albums`** table but keeps its structure intact for future use. You can still add new albums to it or modify its structure later.

One significant aspect of **`DELETE`** operations is that they are logged row by row in the database's transaction log. This means each row deletion is recorded, allowing for the possibility to undo the deletions if the operation is part of a transaction. However, this logging can make **`DELETE`** operations slower when dealing with a large number of rows.

The **`DROP TABLE`** command, in contrast, is much more drastic. When you execute a **`DROP TABLE`** statement, you remove the entire table from the database. This includes not just the data but the table's structure, its columns, indexes, and any constraints defined on it.

For instance, if you decide that the **`artists`** table is no longer needed and you want to erase it entirely from the database, you would use:

```sql
DROP TABLE Artists;
```

Executing this command means the **`artists`** table is deleted. The table, along with all its data and structure, is permanently removed from the database. Unlike **`DELETE`**, **`DROP TABLE`** does not log individual row deletions because it doesn't process each row; it removes the table as a whole. This makes **`DROP TABLE`** a fast operation but with the significant caveat that it is typically irreversible through standard SQL commands. Once a table is dropped, you cannot simply undo the action unless you have backups or specific database recovery tools in place.

## Introduction to Soft Delete

In database management, there are situations where you may want to keep records even after they are marked as deleted. Instead of permanently removing data from the database, you can implement a "soft delete" approach. Soft delete involves adding a column to the table that indicates whether a record is active or deleted, allowing you to retain historical data while still being able to filter out deleted records when querying the table.

Let's explore the concept of soft delete using our example of the terrible trio artist and their albums.

### Adding a Soft Delete Column

To implement soft delete, we need to add a column to the `Artist` and `Album` tables that represents the deleted status of each record. We'll call this column `IsDeleted` and set its default value to `0` (indicating an active record)

In [78]:
%%sql
ALTER TABLE Artist ADD COLUMN IsDeleted INTEGER DEFAULT 0;
ALTER TABLE Album ADD COLUMN IsDeleted INTEGER DEFAULT 0;

 * sqlite:///greatest.db
Done.
Done.


[]

These statements add the `IsDeleted` column to the `Artist` and `Album` tables, respectively.

### Soft Deleting Records

Now, let's say we want to soft delete the artist "The Terrible Trio" and their albums. Instead of using the `DELETE` statement, we'll update the `IsDeleted` column to mark the records as deleted.

In [79]:
%%sql
UPDATE Artist SET IsDeleted = 1 WHERE Name = 'The Terrible Trio';
UPDATE Album SET IsDeleted = 1 WHERE ArtistID = (SELECT ArtistID FROM Artist WHERE Name = 'The Terrible Trio');

 * sqlite:///greatest.db
1 rows affected.
2 rows affected.


[]

The first statement marks the artist "The Terrible Trio" as deleted by setting the `IsDeleted` column to `1`. The second statement marks all the albums associated with "The Terrible Trio" as deleted by setting their `IsDeleted` column to `1`.

### Querying Soft Deleted Records

When querying the `Artist` and `Album` tables, you can filter out the soft deleted records by adding a condition to check the `IsDeleted` column.

In [80]:
%%sql
SELECT * FROM Artist WHERE IsDeleted = 0;

 * sqlite:///greatest.db
Done.


ArtistID,Name,Country,Founded,IsDeleted
1,Marvin Gaye,United States,1939,0
2,The Beach Boys,United States,1961,0
3,Joni Mitchell,Canada,1964,0
4,Stevie Wonder,United States,1961,0
5,Nirvana,United States,1987,0
6,Fleetwood Mac,United Kingdom,1967,0
7,Prince,United States,1975,0
8,Bob Dylan,United States,1961,0
9,Lauryn Hill,United States,1988,0
10,The Beatles,United Kingdom,1960,0


### Restoring Soft Deleted Records
One of the advantages of soft delete is the ability to restore deleted records if needed. To restore a soft deleted record, you can update the IsDeleted column back to 0.

In [ ]:
%%sql
UPDATE Artist SET IsDeleted = 0 WHERE Name = 'The Terrible Trio';
UPDATE Album SET IsDeleted = 0 WHERE ArtistID = (SELECT ArtistID FROM Artist WHERE Name = 'The Terrible Trio');

These statements restore the artist "The Terrible Trio" and their associated albums by setting the `IsDeleted` column back to `0`.

### Considerations

When implementing soft delete, keep in mind the following considerations:

-   Soft delete adds an overhead to your queries, as you need to include the `IsDeleted` condition in your `WHERE` clauses to filter out deleted records.
-   Soft deleted records still occupy space in the database, so you need to have a strategy for eventually purging them if necessary.
-   If you have foreign key constraints, you may need to handle soft deletion carefully to maintain data integrity. You can consider using `ON DELETE SET NULL` or `ON UPDATE CASCADE` to manage the relationships between soft deleted records.

Soft delete provides a flexible approach to data deletion, allowing you to retain historical data while still being able to manage deleted records effectivel